# Analysis of the Mutuals Funds from Schwab

### Author: Jefferson Bien-Aimé
### Date : December 8th, 2025

## Executive Summary

## Table of Contents

1. Introduction
2. Problem Statement / Research Question
3. Data Description
4. Setup and Environment
5. Data Loading
6. Data Preparation
7. Model Planning
8. Model Building / Analysis
9. Discussion & Interpretation
10. Conclusion
11. References
12. Appendix

## Introduction

## Problem Statement / Research Question

## Data Description

## Setup and Environment

In [1]:
!pip install google-cloud-storage

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
import os
from google.cloud import storage
from io import BytesIO

## Data Loading

### Functions

In [14]:
from google.cloud import storage

def list_public_bucket(bucket_name, course, project):
    prefix = f"{course}/Project {project}/"

    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blobs = client.list_blobs(bucket, prefix=prefix)
    print(f"Listing files in public bucket '{bucket_name}' under '{prefix}':")
    filenames = []
    for blob in blobs:
        print(blob.name)
        filenames.append(blob.name)

    return filenames

In [18]:
from google.cloud import storage
import pandas as pd
from io import BytesIO
import os
import json

def gcs_file_to_dataframe(bucket_name: str, blob_path: str):

    # Anonymous client for public buckets
    client = storage.Client.create_anonymous_client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_path)

    # Download file content as bytes
    data = blob.download_as_bytes()

    # Detect file extension
    _, ext = os.path.splitext(blob_path.lower())

    if ext == ".csv":
        df = pd.read_csv(BytesIO(data))

    elif ext in [".xls", ".xlsx"]:
        df = pd.read_excel(BytesIO(data))

    elif ext == ".parquet":
        df = pd.read_parquet(BytesIO(data))

    elif ext == ".json":
        # Convert bytes → Python object
        text = data.decode("utf-8")
        json_obj = json.loads(text)

        # If JSON is a list → NORMAL DataFrame
        if isinstance(json_obj, list):
            df = pd.DataFrame(json_obj)

        # If JSON is a dict → flatten into rows
        elif isinstance(json_obj, dict):
            df = pd.json_normalize(json_obj)

        else:
            raise ValueError("Unsupported JSON structure")

    else:
        raise ValueError(f"Unsupported file type: {ext}")

    return df

In [15]:
# Example
BUCKET_NAME = "msba-online-data"
PROJECT_NUMBER = "05"
COURSE = "CIS9650"
salaries_raw = list_public_bucket(BUCKET_NAME,COURSE,PROJECT_NUMBER)

Listing files in public bucket 'msba-online-data' under 'CIS9650/Project 05/':
CIS9650/Project 05/
CIS9650/Project 05/response_0023e4c1-b77e-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_018aa811-b77b-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_019321a1-b77d-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_01d12a21-b77f-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_01df9b51-b77a-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_02546b41-b77c-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_0341eda1-b77e-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04a44421-b77b-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04a606f1-b77d-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04f3c6e1-b77f-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_04fcb9d1-b77a-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_05711491-b77c-11f0-a457-2128cc60a8f4.json
CIS9650/Project 05/response_0661a431-b77e-11f0-a457-2128cc60a

In [21]:
for element in salaries_raw[1:]:
    df = gcs_file_to_dataframe(BUCKET_NAME,  element)
    print(df.head())
    break

   total  hidden                                               rows
0    250       0  [{'uuid': '406bc2ba-a769-45f9-bbbb-73af0bfef9f...


In [1]:
df_expanded = df.explode("details", ignore_index=True)

# 2. Convert the dict into columns
detail_cols = df_expanded["rows"].apply(pd.Series)

# 3. Merge back and drop the dict column
df_flat = pd.concat([df_expanded.drop(columns=["rows"]), detail_cols], axis=1)

df_flat

NameError: name 'df' is not defined

## Data Preparation

## Model Planning

## Model building / Analysis

## Discussion and Results

## Conclusion

## References

1. Barber, David. Bayesian Reasoning and Machine Learning. Cambridge University Press, 2012.
2. Aste, Tomaso, Paola Cerchiello, and Roberta Scaramozzino. "Information-Theoretic Causality Detection between Financial and Sentiment Data."Entropy, vol. 24, no. 6, 2022, pp. 1–18. DOI:10.3390/e24060774.
3. Metz, Cade. "Microsoft Puts OpenAI’s Sam Altman in Charge of New Advanced AI Research Team."
The New York Times, 20 Nov. 2023, www.nytimes.com/2023/11/20/technology/openai-microsoft-altman.html

## Appendix